In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NM_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9539388701,NM,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,3.504900e+14
1,6755517109,NM,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,3.500500e+14
2,8818167010,NM,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,3.500100e+14
3,9499057008,NM,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,3.500100e+14
4,9341497207,NM,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,3.500100e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9539388701,NM,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,3.504900e+14
1,6755517109,NM,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,3.500500e+14
2,8818167010,NM,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,3.500100e+14
3,9499057008,NM,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,3.500100e+14
4,9341497207,NM,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,3.500100e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
94,5798917705,NM,"50 LOS BANOS DR, OJO CALIENTE, NM",21.878201,-102.673121,NaN
164,9950078310,NM,"50 Los Banos Dr, Ojo Caliente, NM",21.878201,-102.673121,NaN
755,3817587100,NM,"103 MAGUEY CT, SUNLAND PARK, NM",31.833009,-106.536423,NaN
1379,9511588504,NM,"761 ST CLOUD RD, WINSTON, NM",34.086180,-118.441194,NaN
1559,1930747201,NM,"HWY 84 CR 0324 #14, TIERRA AMARILLA, NM",34.380289,-103.005229,NaN
3158,5235107705,NM,"4118 CENTRAL AVE SE STE C, ALBUQUERQUE, NM",35.078777,-106.597809,NaN
3338,3006758400,NM,"3316 E Historic Highway 66, Gallup, NM",35.529839,-108.682520,NaN
3440,8029977308,NM,"100 FRONTERA BLVD, JOGUES, NM",34.972730,-105.032364,NaN
4240,5019727208,NM,"4901 Edith Blvd NE, SANDIA PARK, NM",35.129722,-106.631494,NaN
4388,2328867106,NM,"1510 Latigo Circle, ROSWELL, NM",33.435679,-104.544271,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('35')] #NM
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/10 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

10


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13489/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
94,5798917705,NM,"50 LOS BANOS DR, OJO CALIENTE, NM",21.878201,-102.673121,[]
164,9950078310,NM,"50 Los Banos Dr, Ojo Caliente, NM",21.878201,-102.673121,[]
755,3817587100,NM,"103 MAGUEY CT, SUNLAND PARK, NM",31.833009,-106.536423,[]
1379,9511588504,NM,"761 ST CLOUD RD, WINSTON, NM",34.086180,-118.441194,[]
1559,1930747201,NM,"HWY 84 CR 0324 #14, TIERRA AMARILLA, NM",34.380289,-103.005229,[]
3158,5235107705,NM,"4118 CENTRAL AVE SE STE C, ALBUQUERQUE, NM",35.078777,-106.597809,[350010005041009]
3338,3006758400,NM,"3316 E Historic Highway 66, Gallup, NM",35.529839,-108.682520,[350319456011050]
3440,8029977308,NM,"100 FRONTERA BLVD, JOGUES, NM",34.972730,-105.032364,[350199616004125]
4240,5019727208,NM,"4901 Edith Blvd NE, SANDIA PARK, NM",35.129722,-106.631494,[350010032011010]
4388,2328867106,NM,"1510 Latigo Circle, ROSWELL, NM",33.435679,-104.544271,[350050010023011]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13489/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
94,5798917705,NM,"50 LOS BANOS DR, OJO CALIENTE, NM",21.878201,-102.673121,NaN
164,9950078310,NM,"50 Los Banos Dr, Ojo Caliente, NM",21.878201,-102.673121,NaN
755,3817587100,NM,"103 MAGUEY CT, SUNLAND PARK, NM",31.833009,-106.536423,NaN
1379,9511588504,NM,"761 ST CLOUD RD, WINSTON, NM",34.086180,-118.441194,NaN
1559,1930747201,NM,"HWY 84 CR 0324 #14, TIERRA AMARILLA, NM",34.380289,-103.005229,NaN
3158,5235107705,NM,"4118 CENTRAL AVE SE STE C, ALBUQUERQUE, NM",35.078777,-106.597809,350010005041009
3338,3006758400,NM,"3316 E Historic Highway 66, Gallup, NM",35.529839,-108.682520,350319456011050
3440,8029977308,NM,"100 FRONTERA BLVD, JOGUES, NM",34.972730,-105.032364,350199616004125
4240,5019727208,NM,"4901 Edith Blvd NE, SANDIA PARK, NM",35.129722,-106.631494,350010032011010
4388,2328867106,NM,"1510 Latigo Circle, ROSWELL, NM",33.435679,-104.544271,350050010023011


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
94,5798917705,NM,"50 LOS BANOS DR, OJO CALIENTE, NM",21.878201,-102.673121,NaN
164,9950078310,NM,"50 Los Banos Dr, Ojo Caliente, NM",21.878201,-102.673121,NaN
755,3817587100,NM,"103 MAGUEY CT, SUNLAND PARK, NM",31.833009,-106.536423,NaN
1379,9511588504,NM,"761 ST CLOUD RD, WINSTON, NM",34.086180,-118.441194,NaN
1559,1930747201,NM,"HWY 84 CR 0324 #14, TIERRA AMARILLA, NM",34.380289,-103.005229,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
94,5798917705,NaN
164,9950078310,NaN
755,3817587100,NaN
1379,9511588504,NaN
1559,1930747201,NaN
3158,5235107705,350010005041009
3338,3006758400,350319456011050
3440,8029977308,350199616004125
4240,5019727208,350010032011010
4388,2328867106,350050010023011


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NM_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,590847,9539388701,NM,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,3.504900e+14
1,590848,6755517109,NM,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,3.500500e+14
2,590849,8818167010,NM,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,3.500100e+14
3,590850,9499057008,NM,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,3.500100e+14
4,590851,9341497207,NM,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,3.500100e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,590847,9539388701,NM,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,3.504900e+14,NaN
1,590848,6755517109,NM,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,3.500500e+14,NaN
2,590849,8818167010,NM,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,3.500100e+14,NaN
3,590850,9499057008,NM,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,3.500100e+14,NaN
4,590851,9341497207,NM,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,3.500100e+14,NaN
...,...,...,...,...,...,...,...,...
4477,595324,8805347202,NM,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,3.500100e+14,NaN
4478,595325,9394038307,NM,"407 E Broadway, Farmington, NM",36.727896,-108.201310,3.504500e+14,NaN
4479,595326,9614607003,NM,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,3.500900e+14,NaN
4480,595327,9653187305,NM,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,3.500100e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,590847,9539388701,NM,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,3.504900e+14,NaN,350490004001062.0
1,590848,6755517109,NM,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,3.500500e+14,NaN,350050002013012.0
2,590849,8818167010,NM,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,3.500100e+14,NaN,350010037451033.0
3,590850,9499057008,NM,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,3.500100e+14,NaN,350010032011003.0
4,590851,9341497207,NM,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,3.500100e+14,NaN,350010007151025.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9539388701,NM,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,350490004001062.0
1,6755517109,NM,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,350050002013012.0
2,8818167010,NM,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,350010037451033.0
3,9499057008,NM,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,350010032011003.0
4,9341497207,NM,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,350010007151025.0
...,...,...,...,...,...,...
4477,8805347202,NM,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,350010027001044.0
4478,9394038307,NM,"407 E Broadway, Farmington, NM",36.727896,-108.201310,350450004021007.0
4479,9614607003,NM,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,350090003081035.0
4480,9653187305,NM,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,350010037261003.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
94,5798917705,NM,"50 LOS BANOS DR, OJO CALIENTE, NM",21.878201,-102.673121,NaN
164,9950078310,NM,"50 Los Banos Dr, Ojo Caliente, NM",21.878201,-102.673121,NaN
755,3817587100,NM,"103 MAGUEY CT, SUNLAND PARK, NM",31.833009,-106.536423,NaN
1379,9511588504,NM,"761 ST CLOUD RD, WINSTON, NM",34.086180,-118.441194,NaN
1559,1930747201,NM,"HWY 84 CR 0324 #14, TIERRA AMARILLA, NM",34.380289,-103.005229,NaN


Convert to File

In [19]:
fips_NM = fips_merge_drop

In [20]:
fips_NM.to_csv("../../../data/state_data/geo/geo_fips/23/NM_fips_scraped.csv") 